# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [28]:
from sqlalchemy import create_engine
import pandas as pd
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [12]:
'''# import libraries
import re
import numpy as np
import pandas as pd
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sqlalchemy import create_engine
import pickle
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report'''

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [15]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('ETL_Pipeline_Preparation', con=engine)
X = df.message
Y = df.drop(['id','message','original','genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [16]:
def tokenize(text):
    '''
    takes text input and returns tokenized
    and lemmatized list of words
    in lower case with white space stripped
    '''
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer() 

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [17]:
#initiate pipeline
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(RandomForestClassifier()))]) 

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [18]:
# splitting data
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [19]:
# train pipeline
model = pipeline.fit(X_train, y_train)
model

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [20]:
def report(model, X_test, y_test):
    '''
    takes model and test data and returns
    classification report for predictions
    '''
    y_pred = model.predict(X_test)
    
    for item, col in enumerate(y_test):
        print(col)
        print(classification_report(y_test[col], y_pred[:, item]))

In [21]:
report(model, X_test, y_test)

related
             precision    recall  f1-score   support

        0.0       0.34      0.13      0.19      1538
        1.0       0.77      0.92      0.84      4962
        2.0       0.50      0.02      0.04        52

avg / total       0.67      0.73      0.68      6552

request
             precision    recall  f1-score   support

        0.0       0.84      0.98      0.90      5421
        1.0       0.47      0.08      0.14      1131

avg / total       0.77      0.83      0.77      6552

offer
             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00      6519
        1.0       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6552

aid_related
             precision    recall  f1-score   support

        0.0       0.60      0.83      0.69      3837
        1.0       0.46      0.21      0.29      2715

avg / total       0.54      0.57      0.52      6552

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [22]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f38c7fc42f0>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [23]:
'''
parameters = {'tfidf__use_idf': (True, False), 
              'clf__estimator__max_depth': [2,4],
              'clf__estimator__n_estimators': [10, 100], 
              'clf__estimator__min_samples_split': [2, 3]}

cv = GridSearchCV(pipeline, param_grid=parameters)
'''

"\nparameters = {'tfidf__use_idf': (True, False), \n              'clf__estimator__max_depth': [2,4],\n              'clf__estimator__n_estimators': [10, 100], \n              'clf__estimator__min_samples_split': [2, 3]}\n\ncv = GridSearchCV(pipeline, param_grid=parameters)\n"

In [24]:
parameters = {'clf__estimator__max_depth': [2,4],
              'clf__estimator__n_estimators': [5, 10], 
              'clf__estimator__min_samples_split': [2, 3]}

cv = GridSearchCV(pipeline, param_grid=parameters)

In [25]:
cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__max_depth': [2, 4], 'clf__estimator__n_estimators': [5, 10], 'clf__estimator__min_samples_split': [2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [26]:
%%time
cv.fit(X_train, y_train)
report(cv, X_test, y_test)

related
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00      1538
        1.0       0.76      1.00      0.86      4962
        2.0       0.00      0.00      0.00        52

avg / total       0.57      0.76      0.65      6552

request
             precision    recall  f1-score   support

        0.0       0.83      1.00      0.91      5421
        1.0       0.00      0.00      0.00      1131

avg / total       0.68      0.83      0.75      6552

offer
             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00      6519
        1.0       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6552

aid_related
             precision    recall  f1-score   support

        0.0       0.59      1.00      0.74      3837
        1.0       0.00      0.00      0.00      2715

avg / total       0.34      0.59      0.43      6552

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [29]:
new_pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                         ('svd', TruncatedSVD()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', MultiOutputClassifier(AdaBoostClassifier()))]) 

In [30]:
new_model = new_pipeline.fit(X_train, y_train)
new_model

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [31]:
report(new_model, X_test, y_test)

related
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00      1538
        1.0       0.76      1.00      0.86      4962
        2.0       0.00      0.00      0.00        52

avg / total       0.57      0.76      0.65      6552

request
             precision    recall  f1-score   support

        0.0       0.83      1.00      0.91      5421
        1.0       0.00      0.00      0.00      1131

avg / total       0.68      0.83      0.75      6552

offer
             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00      6519
        1.0       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6552

aid_related
             precision    recall  f1-score   support

        0.0       0.59      1.00      0.74      3837
        1.0       0.14      0.00      0.00      2715

avg / total       0.40      0.58      0.43      6552

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [32]:
new_pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f38c7fc42f0>, vocabulary=None)),
  ('svd', TruncatedSVD(algorithm='randomized', n_components=2, n_iter=5,
          random_state=None, tol=0.0)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encodin

In [47]:
new_parameters = {'svd__n_components': [2,3],
                  'clf__estimator__learning_rate': [0.8, 1.0],
                  'clf__estimator__n_estimators': [50, 75]}

new_cv = GridSearchCV(new_pipeline, param_grid=new_parameters)

In [48]:
new_cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'svd__n_components': [2, 3], 'clf__estimator__learning_rate': [0.8, 1.0], 'clf__estimator__n_estimators': [50, 75]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [49]:
%%time
new_cv.fit(X_train, y_train)
report(new_cv, X_test, y_test)

related
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00      1538
        1.0       0.76      1.00      0.86      4962
        2.0       0.00      0.00      0.00        52

avg / total       0.57      0.76      0.65      6552

request
             precision    recall  f1-score   support

        0.0       0.83      1.00      0.91      5421
        1.0       0.00      0.00      0.00      1131

avg / total       0.68      0.83      0.75      6552

offer
             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00      6519
        1.0       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6552

aid_related
             precision    recall  f1-score   support

        0.0       0.59      1.00      0.74      3837
        1.0       0.18      0.00      0.00      2715

avg / total       0.42      0.58      0.43      6552

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [51]:
with open('model.pkl', 'wb') as f:
    pickle.dump(new_cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.